In [95]:
# Ch03. Detecting features in Images - 3
# Transfer learning

In [96]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
import time
import os

import urllib.request
import zipfile

In [ ]:
weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weights_file = "inception_v3.h5"

# urllib.request.urlretrieve(weights_url,weights_file)

pre_trained_model = InceptionV3(input_shape=(150,150,3),
                                include_top=False,
                                weights=None)

pre_trained_model.load_weights(weights_file)

In [ ]:
pre_trained_model.summary()

In [ ]:
# Freeze the entire layer to prevent retraining
for layer in pre_trained_model.layers:
    layer.trainable = False

In [ ]:
# Set a variable to point at mixed7 layer
# It's not actually "cutting" the layer, but to create a pointer to use later
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape : ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(1024, activation ='relu')(x)
x = tf.keras.layers.Dense(1, activation ='sigmoid')(x)

In [ ]:
model = tf.keras.Model(pre_trained_model.input, x)


In [ ]:
model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001),
              loss = 'binary_crossentropy',
              metrics=['acc'])

In [98]:
training_dir = "horse-or-human/training/"
validation_dir = "horse-or-human/validation/"

In [99]:
train_datagen = ImageDataGenerator(rescale=1./255.,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1.0/255.)

train_generator = train_datagen.flow_from_directory(training_dir,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         class_mode='binary',
                                                         target_size=(150, 150))


Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [100]:
history = model.fit_generator(
            train_generator,
            validation_data=validation_generator,
            epochs=3,
            verbose=1)

c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/3
52/52 [==============================] - 12s 186ms/step - loss: 0.1214 - acc: 0.9494 - val_loss: 0.0247 - val_acc: 0.9883

In [101]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_658 (Conv2D)            (None, 74, 74, 32)   864         ['input_8[0][0]']                
                                                                                                  
 batch_normalization_658 (Batch  (None, 74, 74, 32)  96          ['conv2d_658[0][0]']             
 Normalization)                                                                                   
                                                                                            

In [103]:
# predicting images
fn = 'human01-00.png'
path = 'horse-or-human/training/humans/' + fn
img = image.load_img(path, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

image_tensor = np.vstack([x])
classes = model.predict(image_tensor)
print(classes)
print(classes[0])

if classes[0]>0.5:
    print(fn + " is a human")
else:
    print(fn + " is a horse")

[[1.]]
[1.]
human01-00.png is a human
